<a href="https://colab.research.google.com/github/jugalpanchal/bd-chef/blob/main/spark_sql_etl_airline_anlys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Follow the steps to install the dependencies:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # install java
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz # spark package download
!tar xf spark-3.1.2-bin-hadoop3.2.tgz # unzip spark package
!pip install -q findspark # install spark

# Set the location of Java and Spark:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

# create or get spark session
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Spark_App1") \
        .getOrCreate()

sc = spark.sparkContext # create a SQL context

In [3]:
from pyspark.sql import SQLContext
from pyspark.sql.types import Row
from datetime import datetime

sqlc = SQLContext(sc) # create a SQL context
sqlc

# Alternative to get the sql method is:
#spark.sql('')

In [4]:
airlines_file_path = 'datasets/airlines.csv'
flights_file_path = 'datasets/flights.csv'
airports_file_path = 'datasets/airports.csv'

In [5]:
airlines_df = spark.read.format('csv')\
  .option('header', 'true')\
  .load(airlines_file_path)

flights_df = spark.read.format('csv')\
  .option('header', 'true')\
  .load(flights_file_path)

airlines_df.show()
flights_df.show()

+-----+--------------------+
| Code|         Description|
+-----+--------------------+
|19031|Mackey Internatio...|
|19032|Munz Northern Air...|
|19033|Cochise Airlines ...|
|19034|Golden Gate Airli...|
|19035|  Aeromech Inc.: RZZ|
|19036|Golden West Airli...|
|19037|Puerto Rico Intl ...|
|19038|Air America Inc.:...|
|19039|Swift Aire Lines ...|
|19040|American Central ...|
|19041|Valdez Airlines: VEZ|
|19042|Southeast Alaska ...|
|19043|Altair Airlines I...|
|19044|Chitina Air Servi...|
|19045|Marco Island Airw...|
|19046|Caribbean Air Ser...|
|19047|Sundance Airlines...|
|19048|Seair Alaska Airl...|
|19049|Southeast Airline...|
|19050|Alaska Aeronautic...|
+-----+--------------------+
only showing top 20 rows

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+-

In [6]:
airlines_df.createOrReplaceTempView('airlines_tbl')
flights_df.createOrReplaceTempView('flights_tbl')

airlines_tbl_df = sqlc.sql('select * from airlines_tbl')
flights_tbl_df = spark.sql('select * from flights_tbl') # alternative way to call the sql method

airlines_tbl_df.show()
flights_tbl_df.show()

+-----+--------------------+
| Code|         Description|
+-----+--------------------+
|19031|Mackey Internatio...|
|19032|Munz Northern Air...|
|19033|Cochise Airlines ...|
|19034|Golden Gate Airli...|
|19035|  Aeromech Inc.: RZZ|
|19036|Golden West Airli...|
|19037|Puerto Rico Intl ...|
|19038|Air America Inc.:...|
|19039|Swift Aire Lines ...|
|19040|American Central ...|
|19041|Valdez Airlines: VEZ|
|19042|Southeast Alaska ...|
|19043|Altair Airlines I...|
|19044|Chitina Air Servi...|
|19045|Marco Island Airw...|
|19046|Caribbean Air Ser...|
|19047|Sundance Airlines...|
|19048|Seair Alaska Airl...|
|19049|Southeast Airline...|
|19050|Alaska Aeronautic...|
+-----+--------------------+
only showing top 20 rows

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+-

In [7]:
airlines_df.count(), flights_df.count()

(1579, 476881)

In [8]:
airlines_tbl_count_df = sqlc.sql('select count(*) from airlines_tbl')
flights_tbl_count_df = spark.sql('select count(*) from flights_tbl') # alternative way to call the sql method

airlines_tbl_count_df.show()
flights_tbl_count_df.show()

airlines_tbl_count_df.collect()[0][0], flights_tbl_count_df.collect()[0][0]

+--------+
|count(1)|
+--------+
|    1579|
+--------+

+--------+
|count(1)|
+--------+
|  476881|
+--------+



(1579, 476881)

In [9]:
# Distance completed by all filghts
total_distance_tbl_df = spark.sql('select distance from flights_tbl')\
  .agg({'distance':'sum'})\
  .withColumnRenamed('sum(distance)', 'total_distance')

total_distance_tbl_df.show() # 3.7 Billions miles

+--------------+
|total_distance|
+--------------+
|  3.79052917E8|
+--------------+



In [10]:
all_delays_2014 = spark.sql("SELECT date, airlines, flight_number, departure_delay " +
    "FROM flights_tbl WHERE departure_delay > 0 and year(date) = 2014") # the year method extracts a year value from the column(e.g. date) value

all_delays_2014.show(5)

+----------+--------+-------------+---------------+
|      date|airlines|flight_number|departure_delay|
+----------+--------+-------------+---------------+
|2014-04-01|   19805|            2|          14.00|
|2014-04-01|   19805|            4|          25.00|
|2014-04-01|   19805|            6|         126.00|
|2014-04-01|   19805|            7|         125.00|
|2014-04-01|   19805|            8|           4.00|
+----------+--------+-------------+---------------+
only showing top 5 rows



In [11]:
delay_per_airline = spark.sql("SELECT airlines, departure_delay FROM flights_tbl")\
                         .groupBy("airlines")\
                         .agg({"departure_delay":"avg"})\
                         .withColumnRenamed("avg(departure_delay)", "departure_delay")

delay_per_airline.orderBy(delay_per_airline.departure_delay.desc()).show(5) # even we can add the orderby in the query.

+--------+------------------+
|airlines|   departure_delay|
+--------+------------------+
|   19393|13.429567657134724|
|   20366|12.296210112379818|
|   19977| 8.818392620527979|
|   20436| 8.716275167785234|
|   20409|  8.31110357194785|
+--------+------------------+
only showing top 5 rows



In [12]:
# Hightest average departure delay
delay_per_airline = spark.sql("SELECT flights_tbl.airlines, airlines_tbl.description, avg(flights_tbl.departure_delay) as avg_delay " + 
                              "FROM airlines_tbl " +
                              "JOIN flights_tbl " + 
                              "ON airlines_tbl.code = flights_tbl.airlines " + 
                              "WHERE flights_tbl.departure_delay > 0 " +
                              "GROUP BY flights_tbl.airlines, airlines_tbl.description " + 
                              "HAVING avg_delay > 5 " +
                              "ORDER BY avg_delay DESC")

delay_per_airline.show(20)

+--------+--------------------+------------------+
|airlines|         description|         avg_delay|
+--------+--------------------+------------------+
|   20366|ExpressJet Airlin...| 43.62689565130059|
|   20409| JetBlue Airways: B6|36.383284457478005|
|   20304|SkyWest Airlines ...| 34.71599972258825|
|   20398|       Envoy Air: MQ|34.388925950633755|
|   21171|  Virgin America: VX| 33.46314325452017|
|   19805|American Airlines...|32.126479289940825|
|   20436|Frontier Airlines...| 32.11989459815547|
|   20355| US Airways Inc.: US|26.050868486352357|
|   19790|Delta Air Lines I...|25.936406295825595|
|   19393|Southwest Airline...|25.670224669212327|
|   19977|United Air Lines ...| 25.07559880239521|
|   19930|Alaska Airlines I...|25.070856178827498|
|   20437|AirTran Airways C...| 21.01605651894669|
|   19690|Hawaiian Airlines...| 19.98108747044917|
+--------+--------------------+------------------+

